In [21]:
# load supported packages
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
import pandas as pd
import numpy as np
np.random.seed(2018)
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error
trainSize=1000
testSize=100
train = pd.read_json('..//data//review_train.json',orient = 'records',lines = True,chunksize=trainSize)
train=next(train)
test=pd.read_json('..//data//review_test.json',orient = 'records',lines = True,chunksize=testSize)
test=next(test)
#test=pd.read_json('..//data//review_test.json',orient = 'records',lines = True)

trainText=train.iloc[:,3]
testText=test.iloc[:,3]
trainLabel=train.iloc[:,2].values
text=np.concatenate([trainText.values,testText.values])

del trainText
del testText
def preprocess(text): # tokenize
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(token)
    return result
reviewTrain = []
for i in text:
    reviewTrain.append(preprocess(i))
del text
text=[" ".join(x) for x in reviewTrain]

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
stopWord=['of','with','at','from','into','during',
               'including','until','till','against','among',
               'throughout','despite','towards','upon','concerning','to','in',
               'for','on','by','about','like','through','over',
               'before','between','after','since','without','under',
               'within','along','following','across','behind',
               'beyond','plus','except','but','up','out','around','down','off','above','near']
tf = TfidfVectorizer(analyzer='word', max_features=10000,stop_words=stopWord ,lowercase = True)

text2vec=  tf.fit_transform(text)

In [23]:
#%%
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error
lr=LogisticRegression(multi_class='multinomial',solver='newton-cg')
#
#
X_train, X_test, y_train, y_test = train_test_split(text2vec[:trainSize,:], trainLabel, test_size=0.3, random_state=0)
lr.fit(X_train,y_train)
pred=lr.predict(X_test)
print("the score is {}".format(mean_squared_error(pred,y_test)))

#%%
lr.fit(text2vec[:trainSize,:],trainLabel)
ylabel=lr.predict(text2vec[trainSize:,:])
ylabel=np.array(ylabel)
id=np.array(range(1,len(ylabel)+1))
#header=np.array([["Id","Expected"]])
y_pred=ylabel.reshape([-1,1])
id=id.reshape([-1,1])
ans=np.hstack((id,y_pred))
with open("TueG1_submmit6.csv", 'wb') as f:
  f.write(b'Id,Expected\n')

  np.savetxt(f,ans,delimiter=",",fmt="%i,%i")

the score is 3.1366666666666667
